In [1]:
from env import CarEnv
from stable_baselines3 import A2C
from DataTracker import DataTracker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pygame 2.5.2 (SDL 2.28.2, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
Your model will be saved as: ./models/state_with_three_wp_and_v_edelta.zip
Fetching Libraries.. Please Wait..


2024-06-08 22:42:51.131325: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-08 22:42:51.717415: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
total_timesteps = 150000  # 300k
learning_rate = 0.0005  # 0.004 (4*10^-3) recommended
ent_coef = 0.01
gamma = 0.99
gae_lambda = 0.95
max_grad_norm = 0.5

First Iteration


In [3]:
env = CarEnv()
data_tracker = DataTracker()
model = A2C(
    "MlpPolicy",
    env,
    policy_kwargs=dict(net_arch=[256, 256]),
    learning_rate=learning_rate,
    ent_coef=ent_coef,
    gamma=gamma,
    gae_lambda=gae_lambda,
    max_grad_norm=max_grad_norm,
    tensorboard_log="a2c_logs",
)

try:
    model.learn(int(2e3), callback=data_tracker)
except KeyboardInterrupt:
    print("Saving model due to KeyboardInterrupt")
finally:
    model.save("models/a2c_1")
    data_tracker.save("metrics/a2c/a2c_1.csv")

Environment loading..

Observation space:
Box(-inf, inf, (8,), float16)

Action space:
Discrete(5)

Action space sample:
2


/home/keith/.local/lib/python3.10/site-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(
/home/keith/.local/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.reward to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.reward` for environment variables or `env.get_wrapper_attr('reward')` that will search the reminding wrappers.
  logger.warn(
/home/keith/.local/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.obs to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.obs` for environment variables or `env.get_wrapper_attr('obs')` 

In [ ]:
df = pd.read_csv("metrics/a2c_logs.csv")

# plot         self.data = pd.DataFrame(columns=["episode", "avg_reward", "avg_ey", "avg_ephi", "avg_v", "episode_time"])
df.plot(x="episode", y=["avg_reward"], style=".")

# plot just episode time
df.plot(x="episode", y="episode_time")

In [ ]:
# Test the model
model = A2C.load("models/a2c_1")
env = CarEnv()

while True:
    done = truncated = False
    obs = env.reset()
    while not (done or truncated):
        action, _states = model.predict(obs, deterministic=False)
        obs, reward, done, info = env.step(action)
        env.render()

Second Iteration


In [ ]:
model_path = "models/a2c_1"
new_model_path = "models/a2c_2"

env = CarEnv()
data_tracker = DataTracker()
model = A2C.load(model_path, env=env)

try:
    model.learn(int(2e5), callback=data_tracker)
except KeyboardInterrupt:
    print("Saving model due to KeyboardInterrupt")
finally:
    model.save(new_model_path)
    data_tracker.save("metrics/a2c/a2c_2.csv")

In [ ]:
model = A2C.load("models/a2c_2")
env = CarEnv()

while True:
    done = truncated = False
    obs = env.reset()
    while not (done or truncated):
        action, _states = model.predict(obs, deterministic=False)
        obs, reward, done, info = env.step(action)
        env.render()